In [1]:
# ########################################################## #
#                                                            #
# Name: KEV:Constant Evaluator                               #
# Author: GGamov                                             #
# Date: 2019                                                 #
#                                                            #
# ########################################################## #

# import libraries -------------------------------------------

import pandas as pd
from openpyxl import load_workbook
import re
import io
import os

# basic input ------------------------------------------------
 
def eq_scripts_load(_sep, _subdir, _file):
    
    # if specific file selected it should be XLSX one
    if _file != "":
        
        if _subdir != '':
            _subdir = '/' + _subdir
        _subdir = '../../input' + _subdir + '/'

        _file = _subdir + _file
        
        # open excel file
        with open(_file, "rb") as f:
            inmemory_file = io.BytesIO(f.read())
        wb = load_workbook(inmemory_file, read_only = True)
        
        # read data
        r = re.compile(r'^(input\_)*stoich(iometric)*\_coefficients*$')
        st_coeff_data = pd.read_excel(_file, sheet_name = list(filter(r.search, wb.sheetnames))[0])
        
        r = re.compile(r'^(input\_)*concentrations*$')
        con_data = pd.read_excel(_file, sheet_name = list(filter(r.search, wb.sheetnames))[0], header = 1)
        
        r = re.compile(r'^(input\_)*concentrations*$')
        type_con = pd.read_excel(_file, sheet_name = list(filter(r.search, wb.sheetnames))[0]
                                 , header = None, nrows = 1).iloc[0,:]
        
        r = re.compile(r'^(input\_)*k\_constants\_log10$')
        lg_k_data = pd.read_excel(_file, sheet_name = list(filter(r.search, wb.sheetnames))[0])
        
        r = re.compile(r'heats')
        heats_data = pd.read_excel(_file, sheet_name = list(filter(r.search, wb.sheetnames))[0])
        
        r = re.compile(r'targets')
        targets = pd.read_excel(_file, sheet_name = list(filter(r.search, wb.sheetnames))[0], header = None)
        
        r = re.compile(r'enthalpies')
        delta_H = pd.read_excel(_file, sheet_name = list(filter(r.search, wb.sheetnames))[0])
        
    # use a bunch of plain text files instead
    else:
          
        if _subdir != '':
            _subdir = '/' + _subdir
        _subdir = '../../input' + _subdir + '/'

        file_names = list(os.listdir(path = _subdir))

        r = re.compile(r'^(input\_)*stoich(iometric)*\_coefficients*')
        file = list(filter(r.search, file_names))[0]
        file = _subdir + str(file)
        st_coeff_data = pd.read_csv(file, sep = _sep)

        r = re.compile(r'^(input\_)*k\_constants\_log10')
        file = list(filter(r.search, file_names))[0]
        file = _subdir + str(file)
        lg_k_data = pd.read_csv(file, sep = _sep, decimal = ",")
        
        r = re.compile(r'^(input\_)*concentrations*')
        file = list(filter(r.search, file_names))[0]
        file = _subdir + str(file)
        con_data = pd.read_csv(file, sep = _sep, decimal = ",", header = 1)
        
        type_con = pd.read_csv(file, sep = _sep, header = None, nrows = 1).iloc[0,:]
        
        r = re.compile(r'heats')
        file = list(filter(r.search, file_names))[0]
        file = _subdir + str(file)
        heats_data = pd.read_csv(file, sep = _sep, decimal = ",")
        
        r = re.compile(r'targets')
        file = list(filter(r.search, file_names))[0]
        file = _subdir + str(file)
        targets = pd.read_csv(file, sep = _sep, header = None)
        
        r = re.compile(r'enthalpies')
        file = list(filter(r.search, file_names))[0]
        file = _subdir + str(file)
        delta_H = pd.read_csv(file, sep = _sep)

    return st_coeff_data, lg_k_data, con_data, type_con, heats_data, targets, delta_H 

In [2]:
# ########################################################## #
#                                                            #
# Name: KEV:Constant Evaluator                               #
# Author: GGamov                                             #
# Date: 2019                                                 #
#                                                            #
# ########################################################## #

# import libraries -------------------------------------------

import numpy as np
import re

# basic preprocessing ----------------------------------------
    
def eq_preproc(st_coeff_data, lg_k_data, con_data, type_con, heats_data, targets, delta_H):
    
    # checking if there are several series
    
    if 'series' not in con_data.columns:        
        con_data['series'], type_con[np.shape(st_coeff_data)[1]] = '', ''
    
    if 4 not in heats_data.index:
        heats_data.loc[4] = ''
        print(heats_data)

    # series variables
    
    ser_info = con_data['series'].to_numpy()
    ser_unique = np.unique(ser_info)
    ser_num = np.shape(np.unique(ser_info))[0]

    # matrix of stoich coeff with formal reactions added
    st_coeff_matrix = st_coeff_data.drop('name', axis = 1).to_numpy()
    formal_matrix = np.eye(np.shape(st_coeff_matrix)[1], dtype = int)
    st_coeff_matrix = np.vstack((formal_matrix, st_coeff_matrix))
        
    # product names lists : full and base components only
    
    prod_names_con = list(con_data.drop('series', axis = 1))
    prod_names = prod_names_con + st_coeff_data['name'].tolist()
    
    # creating the vector of equilibrium constants including the formal reactions
    lg_k = (np.vstack((np.zeros((np.shape(st_coeff_matrix)[1], 1)), lg_k_data.to_numpy().astype(float))))
    
    # checking the consistency of reagent names in different sheets    
    if prod_names_con != list(st_coeff_data.drop('name', axis = 1)):
        print('Check the consistency of reagent names!')
    
    # split concentrations matrix
    #con_matrix = [g for _, g in con_data.groupby(['series'])]
        
    #for cnm_index, cnm in enumerate(con_matrix):
        #con_matrix[cnm_index] = cnm.drop('series', axis = 1).to_numpy().astype(float)
    con_matrix = con_data.drop('series', axis = 1).to_numpy()
    ser_counts = con_data.groupby(['series']).size().tolist();
    
    # creating vector of indices of components with predetermined concentrations
    ign_indices = np.array(type_con.index[type_con == 'eq'])
    
    # reading volumes from experimental data
    volumes = heats_data.drop('data', axis = 1).to_numpy()[0]
    
    # reading exp heats from experimental data
    heats = heats_data.drop('data', axis = 1).to_numpy()[1] - heats_data.drop('data', axis = 1).to_numpy()[2]
        
    devs = heats_data.drop('data', axis = 1).to_numpy()[3]
    
    # number of constant to find
    tar_names = set(targets.to_numpy()[0][1:])
    tar_num = [index for index, item in enumerate(prod_names) if item in tar_names]
    
    # number of enthalpy to find
    dH_names = np.hstack((prod_names_con, np.transpose(delta_H.drop('Value', axis = 1).to_numpy())[0]))
    un_el = set(prod_names) - set(dH_names)
    dH_ind = [index for index, item in enumerate(prod_names) if item in un_el]
    
    return ser_num, st_coeff_matrix, prod_names, lg_k, prod_names_con, con_matrix, ign_indices, ser_counts, ser_info, type_con, volumes, heats, devs, tar_num, dH_ind  
    # ser_num, ser_counts, ser_info not further used yet! 

In [3]:
# ########################################################## #
#                                                            #
# Name: KEV:Constant Evaluator                               #
# Author: GGamov                                             #
# Date: 2019                                                 #
#                                                            #
# ########################################################## #

# import libraries -------------------------------------------

import math
import numpy as np
from copy import copy, deepcopy

def eq_calc(max_iter, eps, ser_num, st_coeff_matrix, type_con, lg_k,
            con_matrix, ign_indices, ser_counts, ser_info): # ser_num, ser_counts, ser_info not further used yet!
    
    c_res_out = [0] * len(con_matrix)
    
    for k in range(np.shape(con_matrix)[0]):
                
        reag_eq_con_matrix = deepcopy(con_matrix[k]) # initial estimation of equilibrium concentrations of reagents
        init_conc = deepcopy(con_matrix[k]) # value for residual calculation in inner function
        
        prod_eq_con_matrix = inner_eq_calc(reag_eq_con_matrix, max_iter, lg_k, st_coeff_matrix, init_conc, ign_indices, eps)   
                    
        c_res_out[k] += prod_eq_con_matrix[0]

    return c_res_out

def inner_eq_calc(reag_eq_con_matrix, max_iter, lg_k, st_coeff_matrix, init_conc, ign_indices, eps): 
    
    lg_k, st_coeff_matrix, con_matrix = deepcopy(lg_k), deepcopy(st_coeff_matrix), deepcopy(init_conc)
    
    # if some equilibrium concentrations are set
    if np.shape(ign_indices)[0] > 0:

        range_start = st_coeff_matrix.shape[1]
        range_end = lg_k.shape[0]
        
        lg_k_upd = np.matmul(st_coeff_matrix[range_start:range_end, ign_indices],
          np.log10(init_conc[ign_indices])) #+ lg_k[range_start:range_end]
        
        lg_k[range_start:range_end] = lg_k_upd.reshape(-1, 1) + lg_k[range_start:range_end]
        
        init_conc = np.delete(init_conc, ign_indices, axis = 0)
        reag_eq_con_matrix = np.delete(reag_eq_con_matrix, ign_indices, axis = 0)
        st_coeff_matrix = np.delete(st_coeff_matrix, ign_indices, axis = 1)

    # start of iterative procedure
    for it in range(max_iter):

        # caclulating the equilibrium concentrations of products
        prod_eq_con_matrix = np.exp(np.transpose(np.array(math.log(10) * np.array(lg_k))) +
                                    np.dot(st_coeff_matrix, np.log(reag_eq_con_matrix)))
            
        # calculating the total concentrations of reagents
        reag_tot_con_matrix_calc = np.transpose(np.dot(np.transpose(st_coeff_matrix), np.transpose(prod_eq_con_matrix)))
            
        # calculating the residuals
        g_res = np.array(reag_tot_con_matrix_calc) - np.array(init_conc)
                            
        # calculating the Jacobi matrices
        jac_matrix = np.dot(np.transpose(st_coeff_matrix), (np.array(st_coeff_matrix) * np.transpose(prod_eq_con_matrix)))   
        
        # new estimation of equilibrium concentrations of reagents
        prev = np.log(reag_eq_con_matrix)
        reag_eq_con_matrix = np.exp(prev - np.transpose(np.dot(np.linalg.inv(jac_matrix), np.transpose(g_res))))
        reag_eq_con_matrix = reag_eq_con_matrix[0]
        error = abs(np.log(reag_eq_con_matrix) - prev)
                
        # checking the convergence
        if np.max(error) < eps:
               
            # if some equilibrium concentrations are set
            if np.shape(ign_indices)[0] > 0:
                prod_eq_con_matrix[:, ign_indices] = con_matrix[ign_indices]
                        
            break

    return prod_eq_con_matrix   

In [4]:
# input for xlsx file
_subdir = "calorimetry"
_sep = ";"
_file = "test_3.xlsx"

max_iter, eps = 1000, 0.0000001

st_coeff_data, lg_k_data, con_data, type_con, heats_data, targets, delta_H = eq_scripts_load(_sep, _subdir, _file)

(ser_num, st_coeff_matrix, prod_names, lg_k, prod_names_con, con_matrix, ign_indices, ser_counts,
 ser_info, type_con, volumes, heats, devs, tar_num, dH_ind) = eq_preproc(st_coeff_data, lg_k_data, con_data, type_con, heats_data, 
                                                   targets, delta_H)

c_res_out = eq_calc(max_iter, eps, ser_num, st_coeff_matrix, type_con, lg_k,
            con_matrix, ign_indices, ser_counts, ser_info)

print('\nStoich coeff data')
print(st_coeff_data)

print('\nlg K')
print(lg_k_data)

print('\nconcentrations')
print(con_data)

print('\ntype con')
print(type_con)

print('\nexperimental data')
print(heats_data)

print('\ntargets')
print(targets)

print('\ndelta_H')
print(delta_H)


Stoich coeff data
   H  L  M name
0  1  1  0   HL
1  2  1  0  H2L
2  0  1  1   ML
3  1  1  1  HML

lg K
   lg_k
0  5.35
1  7.34
2  7.44
3  9.47

concentrations
           H             L        M series
0   0.019110  1.000000e-15  0.01486      a
1   0.019650  4.205000e-03  0.01451      a
2   0.019110  1.000000e-15  0.01486      b
3   0.019650  4.172000e-03  0.01451      b
4   0.019110  1.000000e-15  0.01486      c
5   0.019650  4.173000e-03  0.01451      c
6   0.019110  1.000000e-15  0.01486      d
7   0.019650  4.191000e-03  0.01451      d
8   0.002568  1.000000e-15  0.01486      e
9   0.003495  4.183000e-03  0.01451      e
10  0.002568  1.000000e-15  0.01486      f
11  0.003492  4.171000e-03  0.01451      f
12  0.002568  1.000000e-15  0.01486      g
13  0.003494  4.179000e-03  0.01451      g
14  0.002568  1.000000e-15  0.01486      h
15  0.003495  4.182000e-03  0.01451      h

type con
0    tot
1    tot
2    tot
3    NaN
Name: 0, dtype: object

experimental data
          data    1 

In [5]:
print('\nSeries')
print(ser_num)
print(ser_counts)
print(ser_info)

print('\nSt coeff matr')
print(st_coeff_matrix)

print('\nprod names')
print(prod_names)
print(prod_names_con)

print('\nconst')
print(lg_k)

print('\nconcentrations')
print(con_matrix)

print(ign_indices)

print('\ntype con')
print(type_con)

print('\nvolumes')
print(volumes)

print('\nheats')
print(heats)

print('\ndeviations')
print(devs)

print('\nwhat to find')
print(tar_num)
print(dH_ind)


Series
8
[2, 2, 2, 2, 2, 2, 2, 2]
['a' 'a' 'b' 'b' 'c' 'c' 'd' 'd' 'e' 'e' 'f' 'f' 'g' 'g' 'h' 'h']

St coeff matr
[[1 0 0]
 [0 1 0]
 [0 0 1]
 [1 1 0]
 [2 1 0]
 [0 1 1]
 [1 1 1]]

prod names
['H', 'L', 'M', 'HL', 'H2L', 'ML', 'HML']
['H', 'L', 'M']

const
[[0.  ]
 [0.  ]
 [0.  ]
 [5.35]
 [7.34]
 [7.44]
 [9.47]]

concentrations
[[1.911e-02 1.000e-15 1.486e-02]
 [1.965e-02 4.205e-03 1.451e-02]
 [1.911e-02 1.000e-15 1.486e-02]
 [1.965e-02 4.172e-03 1.451e-02]
 [1.911e-02 1.000e-15 1.486e-02]
 [1.965e-02 4.173e-03 1.451e-02]
 [1.911e-02 1.000e-15 1.486e-02]
 [1.965e-02 4.191e-03 1.451e-02]
 [2.568e-03 1.000e-15 1.486e-02]
 [3.495e-03 4.183e-03 1.451e-02]
 [2.568e-03 1.000e-15 1.486e-02]
 [3.492e-03 4.171e-03 1.451e-02]
 [2.568e-03 1.000e-15 1.486e-02]
 [3.494e-03 4.179e-03 1.451e-02]
 [2.568e-03 1.000e-15 1.486e-02]
 [3.495e-03 4.182e-03 1.451e-02]]
[]

type con
0    tot
1    tot
2    tot
3    NaN
Name: 0, dtype: object

volumes
[1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2]

heats
[0 -10.0952 0 -10.4

In [6]:
print('\nEq conc')
print(np.array(c_res_out))


Eq conc
[[1.91100000e-02 7.93893011e-22 1.48600000e-02 3.39642998e-18
  6.34283410e-18 3.24923250e-16 6.65336693e-16]
 [1.68906909e-02 5.18144295e-09 1.03569385e-02 1.95928735e-05
  3.23404122e-05 1.47802615e-03 2.67503539e-03]
 [1.91100000e-02 7.93893011e-22 1.48600000e-02 3.39642998e-18
  6.34283410e-18 3.24923250e-16 6.65336693e-16]
 [1.69112769e-02 5.12080459e-09 1.03894322e-02 1.93871783e-05
  3.20398893e-05 1.46531170e-03 2.65525612e-03]
 [1.91100000e-02 7.93893011e-22 1.48600000e-02 3.39642998e-18
  6.34283410e-18 3.24923250e-16 6.65336693e-16]
 [1.69106529e-02 5.12263563e-09 1.03884475e-02 1.93933949e-05
  3.20489802e-05 1.46569672e-03 2.65585579e-03]
 [1.91100000e-02 7.93893011e-22 1.48600000e-02 3.39642998e-18
  6.34283410e-18 3.24923250e-16 6.65336693e-16]
 [1.68994223e-02 5.15566353e-09 1.03707234e-02 1.95054702e-05
  3.22127857e-05 1.47262991e-03 2.66664668e-03]
 [2.56800000e-03 1.91344302e-21 1.48600000e-02 1.10004526e-18
  2.76061328e-19 7.83130869e-16 2.15491112e-16]
 